Relier chaque adresse a son client (jointure)
Choisir la target nommée class (1:fraud/0:No soucis)
Encoder les données catégoriques (labelEncoder de sklearn)
Entrainer avec un random forest
plotter la feature importance
Conclure (peut on stopper la fraude ? Mise en prod ?)

# TP Fraud

## Librairies

In [11]:
import pandas as pd
import ipaddress as ipa
import pandasql as psql

## Datasets

In [12]:
fraud = pd.read_csv("/home/vincent/Hitema/datasets/Fraud_Data.csv")
ip = pd.read_csv("/home/vincent/Hitema/datasets/IpAddress_to_Country.csv")

## Infos de fraud

In [13]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151112 entries, 0 to 151111
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         151112 non-null  int64  
 1   signup_time     151112 non-null  object 
 2   purchase_time   151112 non-null  object 
 3   purchase_value  151112 non-null  int64  
 4   device_id       151112 non-null  object 
 5   source          151112 non-null  object 
 6   browser         151112 non-null  object 
 7   sex             151112 non-null  object 
 8   age             151112 non-null  int64  
 9   ip_address      151112 non-null  float64
 10  class           151112 non-null  int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 12.7+ MB


In [14]:
fraud.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1
3,150084,2015-04-28 21:13:25,2015-05-04 13:54:50,44,ATGTXKYKUDUQN,SEO,Safari,M,41,3.840542e+09,0
4,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0


## Infos de IP

In [15]:
ip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138846 entries, 0 to 138845
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   lower_bound_ip_address  138846 non-null  float64
 1   upper_bound_ip_address  138846 non-null  int64  
 2   country                 138846 non-null  object 
dtypes: float64(1), int64(1), object(1)
memory usage: 3.2+ MB


In [16]:
ip.head()

,lower_bound_ip_address,upper_bound_ip_address,country
0,16777216.0,16777471,Australia
1,16777472.0,16777727,China
2,16777728.0,16778239,China
3,16778240.0,16779263,Australia
4,16779264.0,16781311,China


## Gestion des valeurs manquantes

In [17]:
fraud = fraud.dropna()
ip = ip.dropna()

## Test d'une utilisation de la lib ipadress pour essayer de convertir nos adresses de float à IPV4 (str)

In [18]:
ipa.ip_address(int(fraud['ip_address'].iloc[0]))

IPv4Address('43.173.1.96')

## Requête SQL incluant la jointure des deux DF (LIMIT à 20 pour limiter le temps d'éxécution)

In [19]:
query = "SELECT * FROM fraud, ip WHERE fraud.ip_address BETWEEN ip.lower_bound_ip_address AND ip.upper_bound_ip_address LIMIT 20;"

## Grâce à la lib Pandasql on passe la requete et on retourne un nouveau DF

In [20]:
data = psql.sqldf(query, locals())
data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,lower_bound_ip_address,upper_bound_ip_address,country
0,22058,2015-02-24 22:55:49,2015-04-18 02:47:11,34,QVPSPJUOCKZAR,SEO,Chrome,M,39,7.327584e+08,0,7.298089e+08,734003199,Japan
1,333320,2015-06-07 20:39:50,2015-06-08 01:38:54,16,EOGFQPIZPYXFZ,Ads,Chrome,F,53,3.503114e+08,0,3.355443e+08,352321535,United States
2,1359,2015-01-01 18:52:44,2015-01-01 18:52:45,15,YSSKYOSJHPPLJ,SEO,Opera,M,53,2.621474e+09,1,2.621440e+09,2621505535,United States
3,221365,2015-07-21 07:09:52,2015-09-09 18:40:53,39,NAUITBZFJKHWW,Ads,Safari,M,45,4.155831e+08,0,4.154982e+08,415629311,United States
4,159135,2015-05-21 06:03:03,2015-07-09 08:05:14,42,ALEYXFXINSXLZ,Ads,Chrome,M,18,2.809315e+09,0,2.809266e+09,2809331711,Canada
